In [1]:
import os
os.chdir("../../../")

import pandas as pd
import re
from striprtf.striprtf import rtf_to_text

In [2]:
filepath = os.getcwd() + "/Factiva-20230724-1711.rtf"
with open(filepath, 'r') as file:
    content = file.read()
    text = rtf_to_text(content)

In [25]:
pattern = r"Document PNGPOS.*?(?=\s|$)"
text_lst = re.split(pattern, text)

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [50]:
ss_df = pd.DataFrame()
for news in text_lst[:-1]:
    news_lst = news.split("All rights reserved.")
    header = news_lst[0]
    header_lst = header.split("\n")
    topic = header_lst[0]
    sample = news_lst[-1].replace("\n", " ").strip()
    
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sample)
    ss_temp = pd.DataFrame(ss, columns=ss.keys(), index=[0])
    date_pattern = r'\b\d{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\b'
    ss_temp["date"] = re.findall(date_pattern, c[0])[0]
    ss_temp["text"] = sample
    ss_df = pd.concat([ss_df, ss_temp], axis=0)

In [48]:
ss_df["date"] = pd.to_datetime(ss_df["date"])
ss_df = ss_df.reset_index().drop("index", axis=1)
ss_df.save

,neg,neu,pos,compound,date,text
0,0.005,0.886,0.109,0.9818,2016-04-12,MILNE Bay Province is an exciting tourism dest...
1,0.038,0.838,0.124,0.9793,2016-04-12,Shareholders of Indochine Mining Limited (the ...
2,0.010,0.875,0.116,0.9904,2016-04-12,Tumarau health centre in the inland Turubu are...
3,0.024,0.821,0.155,0.9922,2016-04-12,THE National Capital District Governor Powes P...
4,0.009,0.923,0.068,0.9186,2016-04-12,ICE making machines and commercial coolers is ...
...,...,...,...,...,...,...
95,0.024,0.844,0.131,0.9860,2016-04-12,THE Bougainville Chocolate Festival date has b...
96,0.014,0.918,0.068,0.9136,2016-04-12,ENGA Province has partnered with Innovative Ag...
97,0.006,0.926,0.068,0.9678,2016-04-12,ISLANDS Petroleum (IP) the exclusive distribut...
98,0.000,0.893,0.107,0.9729,2016-04-12,PAPUA New Guinea and the Solomon Islands have ...
